<a href="https://colab.research.google.com/github/PriyathamVarma/PriyathamVarma/blob/main/chat_bot_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is for creating a NLP chatbot.

In [1]:
# Creating the json file bot responses

In [2]:
import random
import numpy as np
import json
import pickle

import nltk
from nltk.stem import WordNetLemmatizer


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
intents = json.loads(open('/content/sample_data/intents.json').read())

In [6]:
intents

{'intents': [{'patterns': ['Hi',
    'Hey',
    'How are you',
    'My Name is ',
    'You can call me ',
    'My name is ',
    'I am ',
    'call me'],
   'responses': ['Hey, user_name, I am Barney. How can I help? Would you like to know cheapest train ticket or predict delay?',
    'Hello, user_name thanks for visiting.I am Barney, I can help you to find cheapest train ticket or predict delayed trains arrival time. What would you like me to do for you? ',
    'Hi user_name,I am Barney, what can I do for you? Do you want know cheapest train tickets or know train delay',
    'Hi user_name,I am Barney, how can I help? I am trained to help find cheapest train tickets or predict train delayed arrival time'],
   'tag': 'greeting'},
  {'patterns': ['I want to book train ticket',
    'Find me cheapeast ticket available',
    'Cheapest ticket',
    'train reservation',
    'train booking',
    'travel booking',
    'rail reservation',
    'travel reservation',
    'booking',
    'reservation

In [7]:
words = []
classes = []
documents = []
ignore_letters = ['?',',','.','!']

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
from nltk.tokenize import word_tokenize

In [10]:
for intent in intents['intents']:
  print(intent)
  for patterns in intent['patterns']:
    print(patterns)
    word_list = nltk.word_tokenize(patterns)
    words.extend(word_list)
    print(words)
    #documents
    documents.append((word_list,intent['tag']))
    #classes
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'My Name is ', 'You can call me ', 'My name is ', 'I am ', 'call me'], 'responses': ['Hey, user_name, I am Barney. How can I help? Would you like to know cheapest train ticket or predict delay?', 'Hello, user_name thanks for visiting.I am Barney, I can help you to find cheapest train ticket or predict delayed trains arrival time. What would you like me to do for you? ', 'Hi user_name,I am Barney, what can I do for you? Do you want know cheapest train tickets or know train delay', 'Hi user_name,I am Barney, how can I help? I am trained to help find cheapest train tickets or predict train delayed arrival time']}
Hi
['Hi']
Hey
['Hi', 'Hey']
How are you
['Hi', 'Hey', 'How', 'are', 'you']
My Name is 
['Hi', 'Hey', 'How', 'are', 'you', 'My', 'Name', 'is']
You can call me 
['Hi', 'Hey', 'How', 'are', 'you', 'My', 'Name', 'is', 'You', 'can', 'call', 'me']
My name is 
['Hi', 'Hey', 'How', 'are', 'you', 'My', 'Name', 'is', 'You', 'can'

In [11]:
words

['Hi',
 'Hey',
 'How',
 'are',
 'you',
 'My',
 'Name',
 'is',
 'You',
 'can',
 'call',
 'me',
 'My',
 'name',
 'is',
 'I',
 'am',
 'call',
 'me',
 'I',
 'want',
 'to',
 'book',
 'train',
 'ticket',
 'Find',
 'me',
 'cheapeast',
 'ticket',
 'available',
 'Cheapest',
 'ticket',
 'train',
 'reservation',
 'train',
 'booking',
 'travel',
 'booking',
 'rail',
 'reservation',
 'travel',
 'reservation',
 'booking',
 'reservation',
 'From',
 'to',
 'Travelling',
 'From',
 'to',
 'From',
 'London',
 'to',
 'Norwich',
 'london',
 'Norwich',
 'today',
 'tomorrow',
 '/',
 '/',
 '-',
 '-',
 '01-10-2022',
 '01/20/22',
 'Febuary',
 'Feb',
 'Jan',
 'March',
 'Mar',
 'Arp',
 'April',
 'May',
 'June',
 'July',
 'Aug',
 'Sept',
 'September',
 'Oct',
 'October',
 'Nov',
 'November',
 'Dec',
 'December',
 'am',
 'pm',
 '00:00',
 'AM',
 '9:00',
 'PM',
 ':',
 'am',
 ':',
 'pm',
 'Bye',
 'See',
 'you',
 'later',
 'Goodbye',
 'Thanks',
 'Thank',
 'you',
 'That',
 "'s",
 'helpful',
 'Thank',
 "'s",
 'a',
 'lot'

In [12]:
documents

[(['Hi'], 'greeting'),
 (['Hey'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['My', 'Name', 'is'], 'greeting'),
 (['You', 'can', 'call', 'me'], 'greeting'),
 (['My', 'name', 'is'], 'greeting'),
 (['I', 'am'], 'greeting'),
 (['call', 'me'], 'greeting'),
 (['I', 'want', 'to', 'book', 'train', 'ticket'], 'booking'),
 (['Find', 'me', 'cheapeast', 'ticket', 'available'], 'booking'),
 (['Cheapest', 'ticket'], 'booking'),
 (['train', 'reservation'], 'booking'),
 (['train', 'booking'], 'booking'),
 (['travel', 'booking'], 'booking'),
 (['rail', 'reservation'], 'booking'),
 (['travel', 'reservation'], 'booking'),
 (['booking'], 'booking'),
 (['reservation'], 'booking'),
 (['From', 'to'], 'askdate'),
 (['Travelling', 'From', 'to'], 'askdate'),
 (['From', 'London', 'to', 'Norwich'], 'askdate'),
 (['london'], 'askdate'),
 (['Norwich'], 'askdate'),
 (['today'], 'asktime'),
 (['tomorrow'], 'asktime'),
 (['/', '/'], 'asktime'),
 (['-', '-'], 'asktime'),
 (['01-10-2022'], 'asktime'),
 (['01/

In [13]:
classes

['greeting',
 'booking',
 'askdate',
 'asktime',
 'askreturn',
 'goodbye',
 'thanks',
 'predict',
 'Feedback']

#Lemmatization

In [14]:
words_new = []
for words in words:
  words_new.append(words)
  print(words)
  

Hi
Hey
How
are
you
My
Name
is
You
can
call
me
My
name
is
I
am
call
me
I
want
to
book
train
ticket
Find
me
cheapeast
ticket
available
Cheapest
ticket
train
reservation
train
booking
travel
booking
rail
reservation
travel
reservation
booking
reservation
From
to
Travelling
From
to
From
London
to
Norwich
london
Norwich
today
tomorrow
/
/
-
-
01-10-2022
01/20/22
Febuary
Feb
Jan
March
Mar
Arp
April
May
June
July
Aug
Sept
September
Oct
October
Nov
November
Dec
December
am
pm
00:00
AM
9:00
PM
:
am
:
pm
Bye
See
you
later
Goodbye
Thanks
Thank
you
That
's
helpful
Thank
's
a
lot
!
predict
delay
Delay
my
train
is
delayed
,
want
to
know
new
arrival
time
?
train
delay
No
Quit
Exit
Nothing
else
1
2
3
4
5


In [15]:
words_new

['Hi',
 'Hey',
 'How',
 'are',
 'you',
 'My',
 'Name',
 'is',
 'You',
 'can',
 'call',
 'me',
 'My',
 'name',
 'is',
 'I',
 'am',
 'call',
 'me',
 'I',
 'want',
 'to',
 'book',
 'train',
 'ticket',
 'Find',
 'me',
 'cheapeast',
 'ticket',
 'available',
 'Cheapest',
 'ticket',
 'train',
 'reservation',
 'train',
 'booking',
 'travel',
 'booking',
 'rail',
 'reservation',
 'travel',
 'reservation',
 'booking',
 'reservation',
 'From',
 'to',
 'Travelling',
 'From',
 'to',
 'From',
 'London',
 'to',
 'Norwich',
 'london',
 'Norwich',
 'today',
 'tomorrow',
 '/',
 '/',
 '-',
 '-',
 '01-10-2022',
 '01/20/22',
 'Febuary',
 'Feb',
 'Jan',
 'March',
 'Mar',
 'Arp',
 'April',
 'May',
 'June',
 'July',
 'Aug',
 'Sept',
 'September',
 'Oct',
 'October',
 'Nov',
 'November',
 'Dec',
 'December',
 'am',
 'pm',
 '00:00',
 'AM',
 '9:00',
 'PM',
 ':',
 'am',
 ':',
 'pm',
 'Bye',
 'See',
 'you',
 'later',
 'Goodbye',
 'Thanks',
 'Thank',
 'you',
 'That',
 "'s",
 'helpful',
 'Thank',
 "'s",
 'a',
 'lot'

In [16]:
lemmatized_words = [lemmatizer.lemmatize(word) for word in words_new if word not in ignore_letters]

In [17]:
sorted(set(lemmatized_words))

["'s",
 '-',
 '/',
 '00:00',
 '01-10-2022',
 '01/20/22',
 '1',
 '2',
 '3',
 '4',
 '5',
 '9:00',
 ':',
 'AM',
 'April',
 'Arp',
 'Aug',
 'Bye',
 'Cheapest',
 'Dec',
 'December',
 'Delay',
 'Exit',
 'Feb',
 'Febuary',
 'Find',
 'From',
 'Goodbye',
 'Hey',
 'Hi',
 'How',
 'I',
 'Jan',
 'July',
 'June',
 'London',
 'Mar',
 'March',
 'May',
 'My',
 'Name',
 'No',
 'Norwich',
 'Nothing',
 'Nov',
 'November',
 'Oct',
 'October',
 'PM',
 'Quit',
 'See',
 'Sept',
 'September',
 'Thank',
 'Thanks',
 'That',
 'Travelling',
 'You',
 'a',
 'am',
 'are',
 'arrival',
 'available',
 'book',
 'booking',
 'call',
 'can',
 'cheapeast',
 'delay',
 'delayed',
 'else',
 'helpful',
 'is',
 'know',
 'later',
 'london',
 'lot',
 'me',
 'my',
 'name',
 'new',
 'pm',
 'predict',
 'rail',
 'reservation',
 'ticket',
 'time',
 'to',
 'today',
 'tomorrow',
 'train',
 'travel',
 'want',
 'you']

In [18]:
classes = sorted(set(classes))

In [19]:
classes

['Feedback',
 'askdate',
 'askreturn',
 'asktime',
 'booking',
 'goodbye',
 'greeting',
 'predict',
 'thanks']

In [20]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# Machine learning part

In [21]:
train = []
output_empty = [0] *len(classes)

In [22]:
output_empty

[0, 0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
documents

[(['Hi'], 'greeting'),
 (['Hey'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['My', 'Name', 'is'], 'greeting'),
 (['You', 'can', 'call', 'me'], 'greeting'),
 (['My', 'name', 'is'], 'greeting'),
 (['I', 'am'], 'greeting'),
 (['call', 'me'], 'greeting'),
 (['I', 'want', 'to', 'book', 'train', 'ticket'], 'booking'),
 (['Find', 'me', 'cheapeast', 'ticket', 'available'], 'booking'),
 (['Cheapest', 'ticket'], 'booking'),
 (['train', 'reservation'], 'booking'),
 (['train', 'booking'], 'booking'),
 (['travel', 'booking'], 'booking'),
 (['rail', 'reservation'], 'booking'),
 (['travel', 'reservation'], 'booking'),
 (['booking'], 'booking'),
 (['reservation'], 'booking'),
 (['From', 'to'], 'askdate'),
 (['Travelling', 'From', 'to'], 'askdate'),
 (['From', 'London', 'to', 'Norwich'], 'askdate'),
 (['london'], 'askdate'),
 (['Norwich'], 'askdate'),
 (['today'], 'asktime'),
 (['tomorrow'], 'asktime'),
 (['/', '/'], 'asktime'),
 (['-', '-'], 'asktime'),
 (['01-10-2022'], 'asktime'),
 (['01/

In [24]:
for documents in documents:
  bag = []
  word_patterns = documents[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  #output
  output_row = list(output_empty)
  output_row[classes.index(documents[1])] = 1
  train.append([bag,output_row])


In [25]:
random.shuffle(train)
training = np.array(train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [26]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [27]:
train_y

[[1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0,

# Model development

In [28]:
model = Sequential()
model.add(Dense(128,input_shape = (len(train_x[0]),), activation = 'relu' ))

In [29]:
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

In [30]:
model.add(Dense(len(train_y[0]), activation='softmax'))

In [31]:
sgd = SGD(lr = 0.01, decay = 1e-6 , momentum=0.9,nesterov=True)
model.compile(loss="categorical_crossentropy",optimizer=sgd, metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [32]:
"""hist = model.fit(np.array(train_x),np.array(train_y),epochs=100,batch_size = 5, verbose=1)
model.save('DS_Bot.model')"""

"hist = model.fit(np.array(train_x),np.array(train_y),epochs=100,batch_size = 5, verbose=1)\nmodel.save('DS_Bot.model')"

In [33]:
hist = model.fit(np.array(train_x),np.array(train_y),epochs=1000,batch_size = 5, verbose=1)
model.save('DS_Bot.h5',hist)

Epoch 1/1000
15/15 [==============================] - 1s 2ms/step - loss: 2.1782 - accuracy: 0.3243
Epoch 2/1000
15/15 [==============================] - 0s 2ms/step - loss: 2.0957 - accuracy: 0.3378
Epoch 3/1000
15/15 [==============================] - 0s 2ms/step - loss: 2.0382 - accuracy: 0.3378
Epoch 4/1000
15/15 [==============================] - 0s 2ms/step - loss: 2.0018 - accuracy: 0.3378
Epoch 5/1000
15/15 [==============================] - 0s 2ms/step - loss: 1.9848 - accuracy: 0.3378
Epoch 6/1000
15/15 [==============================] - 0s 2ms/step - loss: 1.9974 - accuracy: 0.3378
Epoch 7/1000
15/15 [==============================] - 0s 2ms/step - loss: 1.9869 - accuracy: 0.3378
Epoch 8/1000
15/15 [==============================] - 0s 2ms/step - loss: 1.9817 - accuracy: 0.3378
Epoch 9/1000
15/15 [==============================] - 0s 2ms/step - loss: 1.9837 - accuracy: 0.3378
Epoch 10/1000
15/15 [==============================] - 0s 2ms/step - loss: 1.9961 - accuracy: 0.3378

In [34]:
hist

# Model development

In [35]:
words = pickle.load(open('/content/words.pkl','rb'))
classes = pickle.load(open('/content/classes.pkl','rb'))


In [36]:
!pip install keras

In [37]:
from keras import models
from keras.models import load_model

In [38]:
model = load_model('/content/DS_Bot.h5')

In [39]:
# cleaning the sentence
def clean_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

In [40]:
# bag of words
def bag_of_words(sentence):
  sentence_words = clean_sentence(sentence)
  bag = [0] * len(words)

  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1

  return np.array(bag)     



In [41]:
def predict_class(sentence):
  bagOfWords = bag_of_words(sentence)
  res = model.predict(np.array([bagOfWords]))[0]
  threshold = 0.25

  result = [[i,r] for i,r in enumerate(res) if r> threshold]

  result.sort(key=lambda x:x[1], reverse = True)

  return_list = []
  for r in result:
    return_list.append({'intent':classes[r[0]], 'probability':str(r[1])})
  return return_list
  print(res)

In [42]:
def get_res(intents_list,intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']

  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result    

In [44]:

while True:
  message = input()
  ints = predict_class(message)
  res = get_res(ints,intents)
  print(res)